In [1]:

import json
from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

import configparser
config = configparser.ConfigParser()
config.read('config.ini')
apikey = config['DEFAULT']['ApiKey']
url = config['DEFAULT']['Url']

# local external file, gitignore
# to encrypt apikey

# read into array
# pass array to translate service
# iterate following the original order
# O(2*N)


def translateService(to_translate):
    if (to_translate == ''):
        return ''
    authenticator = IAMAuthenticator(apikey)
    language_translator = LanguageTranslatorV3(
        version='2018-05-01',
        authenticator=authenticator
    )

    language_translator.set_service_url(url)

    language = language_translator.identify(
        to_translate).get_result()
    detected_language = json.loads(json.dumps(language, indent=2))['languages'][0]['language']
    model_used = ''
    if(detected_language == 'en'):
        model_used = 'en-fr'
    else:
        model_used = 'fr-en'
    translation = language_translator.translate(
        text=to_translate,
        model_id=model_used).get_result()
    translated_result = json.loads(json.dumps(translation, indent=2, ensure_ascii=False))['translations'][0]['translation']
    return translated_result

# load document
spark = SparkSession \
    .builder \
    .appName("Test") \
    .getOrCreate()

# create spark dataframe, immutable
df = spark.read \
    .options(header=True, encoding='UTF-8') \
    .csv("test.csv")

# col1
client_id_list = [data[0] for data in df.select((col("client id"))).collect()]
# col2
to_translate_list = [data[0] for data in df.select((col("text entry"))).collect()]
# col3
translated_list = []
for text in to_translate_list:
    translated = translateService(text)
    translated_list.append(translated)
# headers
headers = ["client id", "text entry", "translated entry"]

merged_columns_tuple_list = [(client_id_list[i], to_translate_list[i], translated_list[i]) for i in range(0, len(to_translate_list))]

# parse df_new for output
df_new = spark.createDataFrame(merged_columns_tuple_list, schema = headers)
df_new.show()
# generate as output file
df_new.coalesce(1).write.csv('testoutputspark', header=True)


22/04/25 15:30:02 WARN Utils: Your hostname, LAPTOP-USGLV2R6 resolves to a loopback address: 127.0.1.1; using 172.28.45.209 instead (on interface eth0)
22/04/25 15:30:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/25 15:30:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---------+--------------------+--------------------+
|client id|          text entry|    translated entry|
+---------+--------------------+--------------------+
|        1|Hello, how are yo...|Bonjour, comment ...|
|        2|Bonjour, comment ...|Hello, how are yo...|
+---------+--------------------+--------------------+

